In [1]:
# Import necessary libraries
from sympy import *

# Regula-Falsi and Secant methods

Implement the Regula-Falsi method so that, given an interval $[a, b]$ with $f(a)f(b) < 0$, it finds a root $p \in [a, b]$:

In [2]:
def regula_falsi(f, a, b):
    p = a
    while abs(f.subs(x, p).evalf()) > 0.001:
        if f.subs(x,a).evalf() > 0:
            p = a - ((f.subs(x, a).evalf() * (b - a)) / (f.subs(x, b).evalf() - f.subs(x, a).evalf()))
        else:
            p = b - ((f.subs(x, b).evalf() * (a - b)) / (f.subs(x, a).evalf() - f.subs(x, b).evalf()))

        if f.subs(x, p).evalf() * f.subs(x, a).evalf() < 0:
            b = p
        else:
            a = p
    print('Root found: ',p)
    return p

x = Symbol('x')
f = 3*x + sin(x) - exp(x)
a = 0
b = 0.5
print(f.subs(x,regula_falsi(f, a, b)).evalf())


Root found:  0.360571651383406
0.000375123007953782


Implement the Secant method so that, given two initial values $p_0$ and $p_1$, a root $p$ is returned:

In [3]:
def secant(f, p_0, p_1):
    p = p_0
    aux = p_1
    while abs(f.subs(x, p).evalf()-f.subs(x,aux).evalf()) > 0.001:
        if f.subs(x,p_0).evalf() > 0:
            p = p_0 - ((f.subs(x, p_0).evalf() * (p_1 - p_0)) / (f.subs(x, p_1).evalf() - f.subs(x, p_0).evalf()))
        else:
            p = p_1 - ((f.subs(x, p_1).evalf() * (p_0 - p_1)) / (f.subs(x, p_0).evalf() - f.subs(x, p_1).evalf()))

        if f.subs(x, p).evalf() * f.subs(x, p_0).evalf() < 0:
            aux = p_1
            p_1 = p
        else:
            aux = p_0
            p_0 = p
    print('Root found: ',p)
    return p

x = Symbol('x')
f = 3*x + sin(x) - exp(x)
p_0 = 0
p_1 = 0.5
print(f.subs(x,secant(f,p_0,p_1)).evalf())
f = x**7 -3*x**6 - 12*x**5 + 12*x**4 - 27*x**3 + 34*x**2 - 14*x +18
p_0 = -4
p_1 = -3
print(f.subs(x,secant(f,p_0,p_1)).evalf())

Root found:  0.360436443380589
3.68775986946956e-5
Root found:  -3.18302117724287
0.00196438866572635


# Convergence of the secant method

Program a function which would be able to predict if, given a function $f(x)$ with a root $p \in [p_0-\delta, p_0+\delta]$, if the secant method could be able to find $p$:

**Note**: Remember that you can find the derivatives of different order for any given function $f$ using the using the library scipy. You can follow [this example](https://docs.sympy.org/latest/tutorials/intro-tutorial/calculus.html). Additionaly, you can find if a function $f$ is continuous in a domain using the function [`continuous_domain`](https://docs.sympy.org/latest/modules/calculus/index.html#sympy.calculus.util.continuous_domain) function of scipy. `continuous_domain` returns either an Interval (if continuous in all the domain) or a Union of the Intervals in which it is continuous.

In [4]:
from sympy import Interval
from sympy.calculus.util import continuous_domain

def convergence_secant(f, p_0, delta):
    x = Symbol('x')
    domain = Interval(p_0 - delta, p_0 + delta)
    if continuous_domain(diff(f, x, x), x, domain) == domain and diff(f, x).subs(x, p_0).evalf() != 0 and f.subs(x, p_0).evalf() == 0:
        return True
    return False

x = Symbol('x')
f = x**2 - 4
p_0 = 2
delta = 0.1
print(convergence_secant(f, p_0, delta))


True


Program a function which, given a function $f$ with a root $p$ near a value $p_0$, finds the root in the "most efficient way". That is:

1. Check if the secant method would be guaranteed to convergence to $p$ given two initial estimates $p_0$, $p_1$.
2. If it did, apply the secant method to find $p$.
3. If the convergence is not guaranteed, find an interval $[a, b]$ around $p_0$ where there is a change of sign. 
4. Use the Regula-Falsi method for finding the root $p$ in $[a, b]$. 

In [5]:
def find_interval(f, p_0):
    step = 0.25
    continuous_intervals = continuous_domain(f, x, S.Reals)
    number_to_check = 0
    if continuous_intervals == S.Reals:
        domainAllRealNumbers = True
    else:
        domainAllRealNumbers = False
    a = max(p_0 - step, 0.1)
    b = p_0 + step 
    maxIter = 500
    iteraciones = 0
    while f.subs(x,a).evalf() * f.subs(x,b).evalf() > 0 and iteraciones < maxIter:
        if domainAllRealNumbers or a - step > 0:
            a -= step
        else: 
            a += step
        b += step
        iteraciones += 1 
    if iteraciones == maxIter:
        return None, None
    return a, b

In [6]:
def find_root_efficiently(f, p_0, p_1):
    delta = 10
    if convergence_secant(f, p_0, delta) or convergence_secant(f, p_1, delta):
        p = secant(f, p_0, p_1)
    else: 
        a, b = find_interval(f, p_0)
        if a != None and b != None:
            p = regula_falsi(f, a, b)
        else:
            print('Root not founded')
            p = None
    return p

Use the previous function in order to find roots for the following functions:

**List of functions**:

1. $f_1(x)=2.3 x^5 + 2.3 x^4 + 3.2 x + 3.2$
2. $f_4(x)=e^{-x} - \ln x$, for $x > 0$
3. $f_5(x)=\ln x + x$, for $x > 0$
4. $f_8(x)=\ln x + \sqrt{x}$, for $x > 0$
5. $f_7(x)=(x-1)^2 - \sqrt{x}$, for $x > 0$
6. $f_9(x)=e^x - \sin x$, for $x > 0$

In [7]:
f1 = 2.3*x**5 + 2.3*x**4 + 3.2*x +3.2
p_0 = -5
p_1 = 3
p = find_root_efficiently(f1,p_0,p_1)

Root found:  -0.999818308044073


In [8]:
f4 = exp(-x) - log(x)
p_0 = 1
p_1 = 3
p = find_root_efficiently(f4,p_0,p_1)

Root found:  1.31018253369852


In [9]:
f5 = log(x) + x
p_0 = 1
p_1 = 3
p = find_root_efficiently(f5,p_0,p_1)

Root found:  0.567181397950368


In [10]:
f8 = log(x) + sqrt(x)
p_0 = 1
p_1 = 3
p = find_root_efficiently(f8,p_0,p_1)

Root found:  0.495055310470358


In [11]:
f7 = (x - 1)**2 - sqrt(x)
p_0 = 1
p_1 = 3
p = find_root_efficiently(f7,p_0,p_1)

Root found:  0.275542877558864


In [12]:
f9 = exp(x) - sin(x)
p_0 = 1
p_1 = 3
p = find_root_efficiently(f9,p_0,p_1)

Root found:  -3.18399423809415
